# 6.4 Operations on sets of tuples

Operations on compound sets are, as much as possible, the same as the operations
introduced for simple sets in [Chapter 5](../05/05.md). Sets of pairs, triples, or longer tuples can be
combined with `union`, `inter`, `diff`, and `symdiff`; can be tested by `in` and
`within`; and can be counted with `card`. Dimensions of operands must match appropriately,
so for example you may not form the union of a set of pairs with a set of triples.
Also, compound sets in AMPL cannot be declared as `ordered` or `circular`, and
hence also cannot be arguments to functions like first and next that take only
ordered sets.

Another set operator, `cross`, gives the set of all pairs of its arguments — the cross or
Cartesian product. Thus the set expression
```
ORIG cross DEST
```
represents the same set as the indexing expression `{ORIG,DEST}`, and
```
ORIG cross DEST cross PROD
```
is the same as `{ORIG,DEST,PROD}`.

Our examples so far have been constructed so that every compound set has a domain
within a cross product of previously specified simple sets; `LINKS` lies within `ORIG cross DEST`,
for example, and `ROUTES` within `ORIG cross DEST cross PROD`.
This practice helps to produce clear and correct models. Nevertheless, if you find it
inconvenient to specify the domains as part of the data, you may define them instead
within the model. AMPL provides an iterated `setof` operator for this purpose, as in the
following example:
```
set ROUTES dimen 3;
set PROD = setof {(i,j,p) in ROUTES} p;
set LINKS = setof {(i,j,p) in ROUTES} (i,j);
```
Like an iterated sum operator, `setof` is followed by an indexing expression and an
argument, which can be any expression that evaluates to a legal set member. The argument
is evaluated for each member of the indexing set, and the results are combined into
a new set that is returned by the operator. Duplicate members are ignored. Thus these
expressions for `PROD` and `LINKS` give the sets of all objects `p` and pairs `(i,j)` such
that there is some member `(i,j,p)` in `ROUTES`.

As with simple sets, membership in a compound set may be restricted by a logical
condition at the end of an indexing expression. For example, the multicommodity transportation
model could define
```
set DEMAND = {j in DEST, p in PROD: demand[j,p] > 0};
```
so that `DEMAND` contains only those pairs `(j,p)` with positive demand for product `p` at
destination `j`. As another example, suppose that we also wanted to model transfers of the
products from one origin to another. We could simply define
```
set TRANSF = {ORIG,ORIG};
```
to specify the set of all pairs of members from `ORIG`. But this set would include pairs
like `("PITT","PITT")`; to specify the set of all pairs of different members from
`ORIG`, a condition must be added:
```
set TRANSF = {i1 in ORIG, i2 in ORIG: i1 != i2};
```
This is another case where two different dummy indices, `i1` and `i2`, need to be defined
to run over the same set; the condition selects those pairs where `i1` is not equal to `i2`.

If a set is ordered, the condition within an indexing expression can also refer to the
ordering. We could declare
```
set ORIG ordered;
set TRANSF = {i1 in ORIG, i2 in ORIG: ord(i1) < ord(i2)};
```
to define a "triangular" set of pairs from `ORIG` that does not contain any pair and its
reverse. For example, `TRANSF` would contain either of the pairs `("PITT","CLEV")`
or `("CLEV","PITT")`, depending on which came first in `ORIG`, but it would not contain
both.

Sets of numbers can be treated in a similar way, since they are naturally ordered.
Suppose that we want to accommodate inventories of different ages in the multiperiod
production model of [Figure 4-4](../04/4_3_a_model_of_production_and_transportation.ipynb#fig-4-4), by declaring:
```
set PROD;          # products
param T > 0;       # number of weeks
param A > 0;       # maximum age of inventory
var Inv {PROD,0..T,0..A} >= 0; # tons inventoried
```
Depending on how initial inventories are handled, we might have to include a constraint
that no inventory in period `t` can be more than `t` weeks old:
```
subject to Too_Old
  {p in PROD, t in 1..T, a in 1..A: a > t}: Inv[p,t,a] = 0;
```
In this case, there is a simpler way to write the indexing expression:
```
subject to Too_Old
	{p in PROD, t in 1..T, a in t+1..A}: Inv[p,t,a] = 0;
```

Here the dummy index defined by `t in 1..T` is immediately used in the phrase `a in t+1..A`.
In this and other cases where an indexing expression specifies two or more
sets, the comma-separated phrases are evaluated from left to right. Any dummy index
defined in one phrase is available for use in all subsequent phrases.